# MCMC with Cython

## Performance profiling of Python baseline version

In [ ]:
%matplotlib inline

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm
np.random.seed(123)

## Original implementation

In [ ]:
def sampler(data, samples, mu_init=.5, proposal_width=.5, mu_prior_mu=0, mu_prior_sd=1.):
    mu_current = mu_init
    posterior = [mu_current]
    for i in range(samples):
        # suggest new position
        mu_proposal = norm(mu_current, proposal_width).rvs()

        # Compute likelihood by multiplying probabilities of each data point
        likelihood_current = norm(mu_current, 1).pdf(data).prod()
        likelihood_proposal = norm(mu_proposal, 1).pdf(data).prod()
        
        # Compute prior probability of current and proposed mu        
        prior_current = norm(mu_prior_mu, mu_prior_sd).pdf(mu_current)
        prior_proposal = norm(mu_prior_mu, mu_prior_sd).pdf(mu_proposal)
        
        p_current = likelihood_current * prior_current
        p_proposal = likelihood_proposal * prior_proposal
        
        # Accept proposal?
        p_accept = p_proposal / p_current
        accept = np.random.rand() < p_accept
        
        if accept:
            mu_current = mu_proposal
        
        posterior.append(mu_current)
        
    return posterior

In [ ]:
np.random.seed(123)
data = np.random.randn(20)

In [ ]:
%%time 
np.random.seed(123)
posterior = sampler(data, samples=1500, mu_init=1.0)
posterior

## Convert to log pdfs and log likelihoods

* The likelihood calculation `norm(mu, sigma).pdf(data).prod()` is at a risk of floating point underflow if we compute the product of many values all < 1.
* Switching from the product of probabilites to the sum of log probabilites is a better approach.
* We can also remove the computation of some transcendental functions.
* SciPy has a `norm(mu, sigma).logpdf(data).sum()` that we can use instead.

In [ ]:
def log_sampler(data, samples=4, mu_init=.5, proposal_width=.5, plot=False, mu_prior_mu=0, mu_prior_sd=1.):
    mu_current = mu_init
    posterior = [mu_current]
    for i in range(samples):
        # suggest new position
        mu_proposal = norm(mu_current, proposal_width).rvs()

        # Compute likelihood by adding log probabilities of each data point
        log_likelihood_current = norm(mu_current, 1).logpdf(data).sum()
        log_likelihood_proposal = norm(mu_proposal, 1).logpdf(data).sum()
        
        # Compute prior log probability of current and proposed mu        
        log_prior_current = norm(mu_prior_mu, mu_prior_sd).logpdf(mu_current)
        log_prior_proposal = norm(mu_prior_mu, mu_prior_sd).logpdf(mu_proposal)
        
        log_p_current = log_likelihood_current + log_prior_current
        log_p_proposal = log_likelihood_proposal + log_prior_proposal
        
        # Accept proposal?
        log_p_accept = log_p_proposal - log_p_current
        accept = np.random.rand() < np.exp(log_p_accept)
        
        if accept:
            # Update position
            mu_current = mu_proposal
        
        posterior.append(mu_current)
        
    return posterior

In [ ]:
%%time
np.random.seed(123)
log_posterior = log_sampler(data, samples=1500, mu_init=1.0)

### Verify correctness

In [ ]:
np.allclose(posterior, log_posterior)

### What we did

* Converted all priors, likelihoods, and posteriors to log-versions of same.
* Verified that we get the same results.
* Net speedup: nothing.

## Let's re-implement the `logpdf()` computation

* We saw previously that SciPy's stats computations are less than optimal.
* Let's re-implement the `norm().logpdf().sum()` using numpy array operations only.

$$ \log \left(p(x|\mu, \sigma) \right) = \log\left(\prod_i^n N(x_i|\mu, \sigma) \right) $$

$$ = \log\left(\prod_i^n \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \left( - \frac{(x_i - \mu)^2}{2 \sigma^2} \right) \right) $$

$$ = \frac{-n}{2}\log(2 \pi \sigma^2) - \frac{1}{2 \sigma^2} \sum_i^n (x_i - \mu)^2 $$

In [ ]:
from numpy import pi
def norm_logpdf(mu, sigma, x):
    n = x.shape[0]
    return - n / 2.0 * (np.log(2 * pi) + 2.0 * np.log(sigma)) - (0.5 / sigma**2) * np.sum((x - mu)**2)

In [ ]:
x = np.linspace(-1, 3, 1000)
norm_logpdf(1.0, 1.0, x), norm(1.0, 1.0).logpdf(x).sum()

In [ ]:
%%timeit x=np.linspace(-1, 3, 1000)
norm(1.0, 1.0).logpdf(x).sum()

In [ ]:
%%timeit x=np.linspace(-1, 3, 1000)
norm_logpdf(1.0, 1.0, x)

In [ ]:
def log_sampler_v2(data, samples, mu_init=.5, proposal_width=.5, mu_prior_mu=0, mu_prior_sd=1.):
    mu_current = mu_init
    posterior = [mu_current]
    for i in range(samples):
        # suggest new position
        mu_proposal = np.random.normal(mu_current, proposal_width)

        # Compute likelihood by adding log probabilities of each data point
        log_likelihood_current = norm_logpdf(mu_current, 1, data)
        log_likelihood_proposal = norm_logpdf(mu_proposal, 1, data)
        
        # Compute prior log probability of current and proposed mu
        log_prior_current = norm_logpdf(mu_prior_mu, mu_prior_sd, np.array([mu_current]))
        log_prior_proposal = norm_logpdf(mu_prior_mu, mu_prior_sd, np.array([mu_proposal]))
        
        log_p_current = log_likelihood_current + log_prior_current
        log_p_proposal = log_likelihood_proposal + log_prior_proposal
        
        # Accept proposal?
        log_p_accept = log_p_proposal - log_p_current
        accept = np.random.rand() < np.exp(log_p_accept)
        
        if accept:
            # Update position
            mu_current = mu_proposal
        
        posterior.append(mu_current)
        
    return posterior

In [ ]:
np.random.seed(123)
posterior_0 = log_sampler(data, samples=1500, mu_init=1.0)
np.random.seed(123)
posterior_1 = log_sampler_v2(data, samples=1500, mu_init=1.0)
np.allclose(posterior_0, posterior_1)

In [ ]:
%%time
np.random.seed(123)
log_posterior = log_sampler_v2(data, samples=15000, mu_init=1.0)

In [ ]:
%%prun
log_sampler_v2(data, samples=15000, mu_init=1.0)

```
         375004 function calls in 2.893 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    60000    0.728    0.000    2.121    0.000 <ipython-input-12-75eaa9180db6>:2(norm_logpdf)
    60000    0.581    0.000    1.393    0.000 fromnumeric.py:1710(sum)
        1    0.437    0.437    2.893    2.893 <ipython-input-16-05520a17552a>:1(log_sampler_v2)
    60000    0.323    0.000    0.643    0.000 _methods.py:31(_sum)
    60000    0.319    0.000    0.319    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    60000    0.169    0.000    0.169    0.000 {built-in method builtins.isinstance}
    30000    0.134    0.000    0.134    0.000 {built-in method numpy.core.multiarray.array}
    15000    0.092    0.000    0.092    0.000 {method 'normal' of 'mtrand.RandomState' objects}
    15000    0.069    0.000    0.069    0.000 {method 'rand' of 'mtrand.RandomState' objects}
    15000    0.041    0.000    0.041    0.000 {method 'append' of 'list' objects}
```

### What we did

* Re-implemented the `scipy.stats.norm().logpdf()` in NumPy.
* Verified correctness.
* Performance improvement over baseline: ~50x. 

## This is our starting point for the Cython version

* Next step: create a Cython version of `log_sampler()`, verify correctness.

In [ ]:
%load_ext Cython

In [ ]:
%%cython
# cython: profile=True

import numpy as np
cimport numpy as cnp
from numpy import pi

cdef double norm_logpdf(double mu, double sigma, cnp.ndarray[double] x):
    n = x.shape[0]
    return - n / 2.0 * (np.log(2 * pi) + 2.0 * np.log(sigma)) - (0.5 / sigma**2) * np.sum((x - mu)**2)


def log_sampler_cy(data, samples=4, mu_init=.5, proposal_width=.5, mu_prior_mu=0, mu_prior_sd=1.):
    mu_current = mu_init
    posterior = [mu_current]
    for i in range(samples):
        # suggest new position
        mu_proposal = np.random.normal(mu_current, proposal_width)

        # Compute likelihood by adding log probabilities of each data point
        log_likelihood_current = norm_logpdf(mu_current, 1, data)
        log_likelihood_proposal = norm_logpdf(mu_proposal, 1, data)
        
        # Compute prior log probability of current and proposed mu
        log_prior_current = norm_logpdf(mu_prior_mu, mu_prior_sd, np.array([mu_current]))
        log_prior_proposal = norm_logpdf(mu_prior_mu, mu_prior_sd, np.array([mu_proposal]))
        
        log_p_current = log_likelihood_current + log_prior_current
        log_p_proposal = log_likelihood_proposal + log_prior_proposal
        
        # Accept proposal?
        log_p_accept = log_p_proposal - log_p_current
        accept = np.random.rand() < np.exp(log_p_accept)
        
        if accept:
            # Update position
            mu_current = mu_proposal
        
        posterior.append(mu_current)
        
    return posterior

In [ ]:
np.random.seed(123)
posterior_v2 = log_sampler_v2(data, samples=15000, mu_init=1.0)
np.random.seed(123)
posterior_cy = log_sampler_cy(data, samples=15000, mu_init=1.0)
np.allclose(posterior_v2, posterior_cy)

In [ ]:
%%timeit np.random.seed(123)
posterior_cy = log_sampler_cy(data, samples=15000, mu_init=1.0)

In [ ]:
%%timeit np.random.seed(123)
posterior_cy = log_sampler_v2(data, samples=15000, mu_init=1.0)

### What we did
* Converted `log_sampler_v2()` to Cythonized `log_sampler_cy()`, verified correctness.
* Performance improvement over previous version: negligible.
* Why?
* Next step: focus on `norm_logpdf()`

## Add static types to `norm_logpdf()`

Focusing on `norm_logpdf()`, let's add static typing, explicit loops, and tighten up the computation.

In [ ]:
%%cython -a
# cython: profile=True

from libc.math cimport log as clog, pi as cpi
import numpy as np
cimport numpy as cnp
from numpy import pi

cdef double norm_logpdf(double mu, double sigma, cnp.ndarray[double] x):
    cdef double s = 0.0
    cdef int n = x.shape[0]
    cdef int i
    for i in range(n):
        s += (x[i] - mu)**2
    return - n / 2.0 * (clog(2 * cpi) + 2.0 * clog(sigma)) - (0.5 / sigma / sigma) * s


def log_sampler_cy_v2(data, samples=4, mu_init=.5, proposal_width=.5, mu_prior_mu=0, mu_prior_sd=1.):
    mu_current = mu_init
    posterior = [mu_current]
    for i in range(samples):
        # suggest new position
        mu_proposal = np.random.normal(mu_current, proposal_width)

        # Compute likelihood by adding log probabilities of each data point
        log_likelihood_current = norm_logpdf(mu_current, 1, data)
        log_likelihood_proposal = norm_logpdf(mu_proposal, 1, data)
        
        # Compute prior log probability of current and proposed mu
        log_prior_current = norm_logpdf(mu_prior_mu, mu_prior_sd, np.array([mu_current]))
        log_prior_proposal = norm_logpdf(mu_prior_mu, mu_prior_sd, np.array([mu_proposal]))
        
        log_p_current = log_likelihood_current + log_prior_current
        log_p_proposal = log_likelihood_proposal + log_prior_proposal
        
        # Accept proposal?
        log_p_accept = log_p_proposal - log_p_current
        accept = np.random.rand() < np.exp(log_p_accept)
        
        if accept:
            mu_current = mu_proposal
        
        posterior.append(mu_current)
        
    return posterior

In [ ]:
np.random.seed(123)
posterior_v2 = log_sampler_v2(data, samples=15000, mu_init=1.0)
np.random.seed(123)
posterior_cy = log_sampler_cy_v2(data, samples=15000, mu_init=1.0)
np.allclose(posterior_v2, posterior_cy)

In [ ]:
%%timeit np.random.seed(123)
log_sampler_cy(data, samples=15000, mu_init=1.0)

In [ ]:
%%timeit np.random.seed(123)
log_sampler_cy_v2(data, samples=15000, mu_init=1.0)

In [ ]:
634. / 119.

### What we did
* Added static types to `norm_logpdf()`.
* Performance improvement over previous version: ~5x.
* Next step: remove all Python interaction in `norm_logpdf()`.
* We'll also factor out the usage of the `np.random` module, and add static typing to the sampler.

## Remove all Python interaction in `norm_logpdf()`

Pull out all the stops on `norm_logpdf()`.

In [ ]:
%%cython -a
# cython: profile=True

cimport cython
from libc.math cimport log as clog, pi as cpi, exp as cexp
import numpy as np
cimport numpy as cnp
from numpy import pi

# cython compile-time constant
# http://cython.readthedocs.io/en/latest/src/userguide/language_basics.html#compile-time-definitions
DEF LOG_2_PI = 1.8378770664093453

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double norm_logpdf(double mu, double sigma, cnp.ndarray[double, mode='c'] x):
    cdef int i, n = x.shape[0]
    cdef double s = 0.0
    for i in range(n):
        s += (x[i] - mu) * (x[i] - mu)
    return - 0.5 * n * LOG_2_PI - n * clog(sigma) - s / (2.0 * sigma * sigma)

cdef double sample_norm(double mu, double sigma):
    return np.random.normal(mu, sigma)

cdef bint accept_p(double log_p_accept):
    return np.random.rand() < cexp(log_p_accept)

@cython.boundscheck(False)
@cython.wraparound(False)
def log_sampler_cy_v3(cnp.ndarray[double] data,
                      int samples,
                      double mu_init=.5,
                      double proposal_width=.5,
                      double mu_prior_mu=0,
                      double mu_prior_sd=1.):
    
    cdef:
        double mu_proposal, log_likelihood_current, log_likelihood_proposal
        double log_prior_current, log_prior_proposal
        double log_p_current, log_p_proposal
        double log_p_accept
        bint accept
        double mu_current = mu_init
        list posterior = [mu_current]
        int i
        cnp.ndarray[double, mode='c'] np_buf = np.empty((1,), dtype='f8')
        
        
    for i in range(samples):
        # suggest new position
        mu_proposal = sample_norm(mu_current, proposal_width)

        # Compute likelihood by adding log probabilities of each data point
        log_likelihood_current = norm_logpdf(mu_current, 1, data)
        log_likelihood_proposal = norm_logpdf(mu_proposal, 1, data)
        
        # Compute prior log probability of current and proposed mu
        np_buf[0] = mu_current
        log_prior_current = norm_logpdf(mu_prior_mu, mu_prior_sd, np_buf)
        np_buf[0] = mu_proposal
        log_prior_proposal = norm_logpdf(mu_prior_mu, mu_prior_sd, np_buf)
        
        log_p_current = log_likelihood_current + log_prior_current
        log_p_proposal = log_likelihood_proposal + log_prior_proposal
        
        # Accept proposal?
        log_p_accept = log_p_proposal - log_p_current
        
        if accept_p(log_p_accept):
            # Update position
            mu_current = mu_proposal
        
        posterior.append(mu_current)
        
    return posterior

In [ ]:
%%prun
np.random.seed(123)
posterior_cy = log_sampler_cy_v3(data, samples=15000, mu_init=1.0)

In [ ]:
%%timeit np.random.seed(123)
posterior_cy = log_sampler_cy_v3(data, samples=15000, mu_init=1.0)

In [ ]:
np.random.seed(123)
posterior_v2 = log_sampler_v2(data, samples=15000, mu_init=1.0)
np.random.seed(123)
posterior_v3 = log_sampler_cy_v3(data, samples=15000, mu_init=1.0)
np.allclose(posterior_v2, posterior_v3)

### What we did
* Removed all Python interaction in `norm_logpdf`
  * Used a compile-time-constant.
  * Removed boundschecking, wraparound checking, and use C-division semantics.
* Add static typing to `log_sampler()`.
* Performance improvement over previous version: ~2x.

### Next steps
* Remove Python interaction in `sample_norm()` and `accept_p()`.
* This will demonstrate using an external Cython package.